In [209]:
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import re

# Load Raw Data
inputDF = pd.read_excel('loan_default_data.xlsx')

# Drop irrelevent columns
# 1) Unnamed Column: there are no details provided for these columns
unnamed_col = [i for i in inputDF.columns if 'unnamed' in i.lower()]
inputDF.drop(unnamed_col, axis=1, inplace=True)

# 2) ID Column: IDs are randomly (or increasing continuously) assigned numbers
# it has no correlation to the decision
id_col = ['id', 'member_id']
inputDF.drop(id_col, axis=1, inplace=True)

# 3) zip_code and address_state
# we might be able to include these columns if we can find a suitable way to group the states into fewer groups
# as for now, I will drop this
id_col = ['zip_code', 'address_state']
inputDF.drop(id_col, axis=1, inplace=True)

# 4) loan_status
# i am not quite understand this column, but seems like repay_fail depends on this column
# e.g.: Fully Paid  -> repay_fail False
#       Charged Off -> repay_fail True
# since this columns seems highly correlated to the target variable, I will drop this
inputDF.drop('loan_status', axis=1, inplace=True)

# 5) loan_amount, funded_amount_investors and total_payment_investors
# we only consider funded_amount which is the amount the client received
# Investors should not affect client's repayment
amt_list = ['loan_amount', 'funded_amount_investors', 'total_payment_investors']
inputDF.drop(amt_list, axis=1, inplace=True)

# 6) next_payment_date and months_since_last_delinquency: Too much missing values
inputDF.drop(['next_payment_date','months_since_last_delinquency'], axis=1, inplace=True)

# TEST: remove last_payment_amnt, total_received_interest, total_payment
inputDF.drop(['last_payment_amnt','total_received_interest', 'total_payment'], axis=1, inplace=True)


# function to calculate months between dates
def months_between(start_date, end_date):
    try:
        delta = relativedelta(end_date, start_date)
        return delta.years * 12 + delta.months
    except AssertionError:
        return None


# Modify Data
# 1) change term to int column
inputDF['term'] = inputDF['term'].apply(lambda x: re.search('\d+',x).group()).astype(int)

# 2) change home_ownership value 'NONE' to None
inputDF['home_ownership'] = inputDF['home_ownership'].replace('NONE', None)

# 3) split verification_status to verified and source_verified (bool columns instead of category)
# Not Verified:     verified: False     source_verified: False
# Verified:         verified: True      source_verified: False
# Source Verified:  verified: True      source_verified: True
inputDF['verified'] = inputDF['verification_status'].isin(['Verified', 'Source Verified'])
inputDF['source_verified'] = inputDF['verification_status'].isin(['Source Verified'])
# drop verification_status column
inputDF.drop('verification_status', axis=1, inplace=True)

# 4) Group employment_length into quantiles
# because "more than 10 years" has been grouped into 10+, this is no longer continuous
emp_year = inputDF['employment_length']\
    .dropna()\
    .replace('< 1 year', '0 year')\
    .replace('10+ years', '10 years')\
    .apply(lambda x: int(re.search('\d+', x).group()))
emp_q = emp_year.quantile([0.25,0.50,0.75])
def getEmpLengthGroup(year):
    if year==None:
        return None
    else:
        if year <= emp_q[0.25]:
            return f'0 - {int(emp_q[0.25])}'
        elif year <= emp_q[0.50]:
            return f'{int(emp_q[0.25])+1} - {int(emp_q[0.50])}'
        elif year <= emp_q[0.75]:
            return f'{int(emp_q[0.50])+1} - {int(emp_q[0.75])}'
        else:
            return f'{int(emp_q[0.75])+1}+'
inputDF['employment_length_group'] = emp_year.apply(getEmpLengthGroup)
# drop employment_length column
inputDF.drop('employment_length', axis=1, inplace=True)

# 5) Change repay_fail column dtype to bool
inputDF['repay_fail'] = inputDF['repay_fail'].astype(bool)

# 6) calculate Maturity Date
inputDF['maturity_date'] = inputDF.apply(lambda x: x['issue_date'] + relativedelta(months=x['term']), axis=1)

# 7) calculate Balance in percentage (funded_amount - total_received_principal)
# update: looks like this variable is highly correlated to repay_fail, probably because this is historical data
# if there are balance amount unpaid, it is already a sign of failed repayment
# inputDF['balance_percentage'] = (inputDF['funded_amount'] - inputDF['total_received_principal']) / inputDF['funded_amount']
# inputDF['balance_percentage'] = inputDF['balance_percentage'].apply(lambda x: round(x, 2))
# Dropping total_received_principal as well
inputDF.drop('total_received_principal', axis=1, inplace=True)

# 8) calculate all date columns in terms of months, using last_credit_pull_date as reference date
inputDF['months_to_maturity'] = inputDF\
    .apply(lambda x: months_between(x['last_credit_pull_date'],x['maturity_date']), axis=1)
inputDF.loc[inputDF['months_to_maturity'] < 0, 'months_to_maturity'] = 0
inputDF['months_since_issued'] = inputDF\
    .apply(lambda x: months_between(x['issue_date'],x['last_credit_pull_date']), axis=1)
inputDF['months_since_first_credit'] = inputDF\
    .apply(lambda x: months_between(x['earliest_credit_line'],x['last_credit_pull_date']), axis=1)
inputDF['months_since_last_payment'] = inputDF\
    .apply(lambda x: months_between(x['last_payment_date'],x['last_credit_pull_date']), axis=1)

# 9) few valuse in revolving_utillization is in different format
# Need to convert to float
def convertStrPercentage(val):
    try:
        # NOTE: the double quotation marks is a unique character (” != " and “ != ")
        val = val.replace('”', '').replace('“', '').replace('%', '')
        return float(val)/100
    except:
        return val
inputDF['revolving_utillization'] = inputDF['revolving_utillization'].apply(convertStrPercentage)

# 10) too many purpose categories, map purpose categories into high/low risk
# we can use a boolean column here, using 'high_risk_purpose' column with value True or False
# I dont have reliable data to classify each purpose's risk level,
# So for now I categorize the following purpose as high risk
#   Debt Consolidation and Credit Card:     This usually means the client has huge debt
#   Small Business:                         businesses have probability to flop
#   Medical:                                Applying loans for medical bill might indicate that the client has health issues,
#                                           and possibility of "unable to work" condition
highRiskMap = {
    'major_purchase': False,
    'other': False,
    'debt_consolidation': True,
    'credit_card': True,
    'small_business': True,
    'medical': True,
    'wedding': False,
    'car': False,
    'home_improvement': False,
    'educational': False,
    'vacation': False,
    'house': False,
    'moving': False,
    'renewable_energy': False
}
inputDF['high_risk_purpose'] = inputDF['purpose'].map(highRiskMap)
inputDF.drop('purpose', axis=1, inplace=True)

# TEST remove datetime col
dtcol = ['issue_date','earliest_credit_line','last_payment_date','last_credit_pull_date','maturity_date']
inputDF.drop(dtcol, axis=1, inplace=True)
# TEST remove all month column
mcol = ['months_to_maturity','months_since_issued','months_since_first_credit','months_since_last_payment']
inputDF.drop(mcol, axis=1, inplace=True)

# Drop rows with missing values
inputDF.dropna(inplace=True)

# # One-hot encode categorical columns
inputDF = pd.get_dummies(inputDF, columns=['home_ownership', 'employment_length_group'], drop_first=True)

In [210]:
# We still have a lot of variable, this will check if any variables are correlated
c = inputDF.corr()

# reduce variable that are highly correlated
# funded_amount and installment
inputDF.drop('installment', axis=1, inplace=True)

# no_total_account and no_open_account
inputDF.drop('no_total_account', axis=1, inplace=True)

In [211]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score

# Split the data into features (X) and target variable (y)
X = inputDF.drop('repay_fail', axis=1)
y = inputDF['repay_fail']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Train a logistic regression model
model = LogisticRegression(max_iter=100000)
model.fit(X_train, y_train)

# Test
y_pred = model.predict(X_test)

# Still getting Evalutions Exceeds Limit, max_iter already high, might need to check if any more variable can be dropped
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

[[6327   20]
 [1127   11]]
0.8467601870407482


c:\Users\epulr\Downloads\Python Technical Assessment\Python Technical Assessment\env\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
